In [1]:
!pip install -q stanza
!pip install -q pydrive

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 987.4/987.4 kB 25.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from pathlib import Path
import stanza
from stanza.utils.conll import CoNLL
from io import StringIO
# download and initialize Stanza pipeline
stanza.download("de")
nlp_de = stanza.Pipeline("de", processors="tokenize,mwt,pos,lemma,depparse", use_gpu=True)

stanza.download("en")
nlp_en = stanza.Pipeline("en", processors="tokenize,mwt,pos,lemma,depparse", use_gpu=True)


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: de (German) ...


INFO:stanza:Downloaded file to /root/stanza_resources/de/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: de (German):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package           |
---------------------------------
| tokenize  | combined          |
| mwt       | combined          |
| pos       | combined_charlm   |
| lemma     | combined_nocharlm |
| depparse  | combined_charlm   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [7]:
folder_name = input("Enter folder name: ")
def create_merged_conllu(df, text_col, lang_code, out_dir="conllu_file", folder_name=folder_name,file_name = ""):
    """
    Merge multiple rows of text in a DataFrame into a single .conllu file
    with sequential sent_id and cumulative character offsets.

    Args:
        df: pandas DataFrame containing text data
        text_col: column name containing the text
        lang_code: language code for Stanza ('de', 'en', etc.)
        out_dir: parent folder to save the merged file
    Returns:
        Path to the merged .conllu file
    """
    out_path = Path(out_dir) /  lang_code /folder_name
    out_path.mkdir(parents=True, exist_ok=True)
    merged_file = out_path / f"{file_name}_{lang_code}.conllu"
    if lang_code == "de":
        nlp = nlp_de
    else:
        nlp = nlp_en



    sent_counter = 0
    char_offset = 0

    with open(merged_file, "w", encoding="utf-8") as out_f:
        for i, row in df.iterrows():
            text = str(row[text_col])
            doc = nlp(text)

            s = StringIO()
            CoNLL.write_doc2conll(doc, s)
            conllu_str = s.getvalue()

            new_lines = []
            for line in conllu_str.split("\n"):
                if line.startswith("# sent_id = 0"):
                    line = f"#Seg_id = {sent_counter}\n #sent_id = {sent_counter}"
                    sent_counter += 1
                elif line.startswith("# sent_id = "):
                    line = f"# sent_id = {sent_counter-1}"
                elif line.startswith("# text"):

                    # optional: adjust text comment if needed
                    pass
                elif line and not line.startswith("#"):
                    parts = line.split("\t")
                    misc = parts[-1]
                    new_misc_items = []
                    for item in misc.split("|"):
                        if item.startswith("start_char=0"):
                            orig_start = int(item.split("=")[1])
                            item = f"start_char={char_offset + orig_start}"

                        elif item.startswith("end_char="):
                            orig_end = int(item.split("=")[1])
                            item = f"end_char={char_offset + orig_end}"
                        new_misc_items.append(item)
                    parts[-1] = "|".join(new_misc_items)
                    line = "\t".join(parts)
                new_lines.append(line)


            updated_conllu = "\n".join(new_lines)

            out_f.write(updated_conllu + "\n\n")

            char_offset += len(text) + 1  # space/newline between texts
            #print(f"Processed row {i} ({row.get('id', 'no id')})")

    print(f"Merged file saved at: {merged_file}")
    return merged_file


In [11]:
from google.colab import files
import zipfile
import os

# Step 1: Upload the zip file
uploaded = files.upload()
zip_path = list(uploaded.keys())[0]  # Get the uploaded file name automatically

# Step 2: Extract into a folder named after the zip file (without .zip)
extract_path = os.path.join("/content", os.path.splitext(zip_path)[0])
os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Files extracted to:", extract_path)

# Step 3: Find all .xlsx files recursively
excel_files = []
for root, dirs, files_in_dir in os.walk(extract_path):
    for file in files_in_dir:
        if file.lower().endswith(".xlsx"):
            excel_files.append(os.path.join(root, file))


Saving Art_aligned.zip to Art_aligned (1).zip
✅ Files extracted to: /content/Art_aligned (1)


In [8]:
import pandas as pd
import glob

# Find all Excel files inside the extracted folder
excel_files = glob.glob(os.path.join(extract_path, "*.xlsx")) + \
              glob.glob(os.path.join(extract_path, "*.xls"))

dfs = {}
for f in excel_files:
    print(f"\n📥 Reading {os.path.basename(f)} ...")
    df = pd.read_excel(f, engine="openpyxl")
    dfs[os.path.basename(f)] = df

    # Remove extension for clean name
    file_name = os.path.basename(f).rsplit('_', 1)[0]
    print(file_name)

    # Example: merge your German + English cols
    merged_de_file = create_merged_conllu(df, text_col="de", lang_code="de", file_name=file_name)
    merged_en_file = create_merged_conllu(df, text_col="en", lang_code="en", file_name=file_name)



📥 Reading Art16_aligned.xlsx ...
Art16
Merged file saved at: conllu_file/de/Art/Art16_de.conllu
Merged file saved at: conllu_file/en/Art/Art16_en.conllu

📥 Reading Art10_aligned.xlsx ...
Art10
Merged file saved at: conllu_file/de/Art/Art10_de.conllu
Merged file saved at: conllu_file/en/Art/Art10_en.conllu

📥 Reading Art7_aligned.xlsx ...
Art7
Merged file saved at: conllu_file/de/Art/Art7_de.conllu
Merged file saved at: conllu_file/en/Art/Art7_en.conllu

📥 Reading Art13_aligned.xlsx ...
Art13
Merged file saved at: conllu_file/de/Art/Art13_de.conllu
Merged file saved at: conllu_file/en/Art/Art13_en.conllu

📥 Reading Art14_aligned.xlsx ...
Art14
Merged file saved at: conllu_file/de/Art/Art14_de.conllu
Merged file saved at: conllu_file/en/Art/Art14_en.conllu

📥 Reading Art1_aligned.xlsx ...
Art1
Merged file saved at: conllu_file/de/Art/Art1_de.conllu
Merged file saved at: conllu_file/en/Art/Art1_en.conllu

📥 Reading Art8_aligned.xlsx ...
Art8
Merged file saved at: conllu_file/de/Art/Art8_

In [10]:

!zip -r conllu_file.zip conllu_file

from google.colab import files
files.download("conllu_file.zip")


  adding: conllu_file/ (stored 0%)
  adding: conllu_file/de/ (stored 0%)
  adding: conllu_file/de/Art/ (stored 0%)
  adding: conllu_file/de/Art/Art12_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art3_de.conllu (deflated 81%)
  adding: conllu_file/de/Art/Art11_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art5_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art16_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art6_de.conllu (deflated 81%)
  adding: conllu_file/de/Art/Art10_de.conllu (deflated 80%)
  adding: conllu_file/de/Art/Art8_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art7_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art14_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art1_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art13_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art15_de.conllu (deflated 82%)
  adding: conllu_file/de/Art/Art4_de.conllu (deflated 81%)
  adding: conllu_file/de/Art/Art2_de.conllu (deflate

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>